# Simple ROI Visualization Widget
View weighted ROI "postage stamps" for selection by shape and size.

In [33]:
import os
import sys
import psycopg2
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt
import numpy as np
from slapp.evaluation.views import postage_stamp

### Retrieve ROIs from Database
Note you'll need to have the environment variables "LABELING_USER" and "LABELING_PASSWORD" set to appropriate values.
This will get the ROIs from the latest run of an experiment_id, if there are more than one.

In [29]:
with psycopg2.connect(
        dbname="ophys_segmentation_labeling", 
        user=os.environ["LABELING_USER"], 
        host="aibsdc-dev-db1", 
        password=os.environ["LABELING_PASSWORD"], 
        port=5432) as conn:
    with conn.cursor() as cursor:
        cursor.execute(
            """
            SELECT
                DISTINCT ON (roi_id, experiment_id)
                rois.id as roi_id, coo_data, coo_row, coo_col, video_shape, ophys_experiment_id as experiment_id
            FROM 
                rois
            JOIN 
                segmentation_runs 
            ON 
                rois.segmentation_run_id = segmentation_runs.id
            ORDER BY ophys_experiment_id, roi_id, segmentation_runs.created_at DESC
            """)
        cols = list(map(lambda x: x[0], cursor.description))
        result = pd.DataFrame(cursor.fetchall(), columns=cols).set_index("roi_id")

rois_per_exp = result.reset_index().groupby("experiment_id")["roi_id"].apply(list).to_dict()

### Generate Images
Make the postage stamp image arrays. Some artifacts will have ROIs larger than 100x100 pixels -- these will show a warning and will not be padded.

In [34]:
def _make_stamps(coo_data, coo_row, coo_col, shape):
    mat = coo_matrix((coo_data, (coo_row, coo_col)), shape=tuple(shape))
    return postage_stamp(mat, canvas_size=(100,100))

result["stamp"] = result[["coo_data", "coo_row", "coo_col", "video_shape"]].apply(lambda x: _make_stamps(*x), axis=1)

### Display Widget
Select by experiment_id and roi_id dropdowns. Choosing a different experiment_id will refresh the roi_id dropdown.

In [35]:
@interact
def show_postage(exp_id=rois_per_exp):
    @interact
    def _show_postage(roi_id=exp_id):
        roi = result.loc[roi_id]["stamp"]
        plt.imshow(roi)

interactive(children=(Dropdown(description='exp_id', options={716956096: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1…